In [10]:
import numpy as np
import math
from scipy.interpolate import interp1d

import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

from numpy.fft import fft, ifft , rfft, irfft , fftfreq
from numpy import exp, log, log10, cos, sin, pi, cosh, sinh , sqrt
from classy import Class
from scipy.optimize import fsolve
from scipy.special import gamma
from scipy.special import hyp2f1
import sys,os
from time import time
from scipy.integrate import quad
import scipy.integrate as integrate
from scipy import special
from scipy.special import factorial

In [11]:
#Starting CLASS
z_pk = 0.61
common_settings = {# fixed parameters
                   'A_s':2.089e-9,
                   'n_s':0.9649,
                   'tau_reio':0.052,
                   'omega_b':0.02237,
                   'h':0.6736,
                   'YHe':0.2425,
#                   'N_eff':3.046,
                   'N_ur':2.0328,
                   'N_ncdm':1,
#                   'N_ncdm':0,
                   'm_ncdm':0.06,
                   # Interacting Dark Matter parameters
                   'sigma_dmeff':1e-25,
                   'omega_cdm':1e-15,
                   'm_dmeff':1.0,
                   'npow_dmeff':0,
                   'Vrel_dmeff':0,
                   'dmeff_target': 'baryons',
                   # other output and precision parameters
                   'P_k_max_1/Mpc':100.0,
                   'z_pk':z_pk,
                   'output':'mPk,tCl,pCl,lCl',
                   'lensing':'yes'}
#                   'z_max_pk':20.,

M = Class()
M.set(common_settings)
#compute linear
M.set({ 'non linear':'no',
        'omega_dmeff':0.120,
      })

M.compute()

M1 = Class()
M1.set(common_settings)
#compute Halofit
M1.set({'non linear':'Halofit',
        'omega_dmeff':0.120,
      })

M1.compute()

#now we compute EFT
M2 = Class()
M2.set(common_settings)
M2.set({'non linear':'PT',
        'IR resummation':'Yes',
        'Bias tracers':'Yes',
        'cb':'No',
        'RSD':'Yes',
        'AP':'Yes',
        'Omfid':'0.31',
        'omega_dmeff':0.120,
       })
M2.compute()

In [14]:
#Extracting and plotting spectra
bg_linear = M.get_background()
th_linear = M.get_thermodynamics()
cl_linear = M.lensed_cl()

bg_Halofit = M1.get_background()
th_Halofit = M1.get_thermodynamics()
cl_Halofit = M1.lensed_cl()

bg_EFT = M2.get_background()
th_EFT = M2.get_thermodynamics()
cl_EFT = M2.lensed_cl()

h = M.h()
k = np.linspace(log(0.0001),log(50),200)
k = np.exp(k)
twopi = 2.*math.pi
khvec = k*h

In [40]:
f,ax_list = plt.subplots(1,3,figsize=(9,3),constrained_layout=True)
ax = ax_list.ravel()

ax[0].set_xlabel('$\ell$')
ax[0].set_ylabel('$C_\ell^{TT}-C_\ell^{TT, linear}/C_\ell^{TT, linear}$')
ax[0].set_xlim([1,2500])

ax[1].set_xlabel('$\ell$')
ax[1].set_ylabel('$C_\ell^{EE}-C_\ell^{EE, linear}/C_\ell^{EE, linear}$')
ax[1].set_xlim([1,2500])

#ax[2].set_xscale('log')
ax[2].set_xlabel('$\ell$')
#ax[2].set_yscale('log')
ax[2].set_ylabel('$C_\ell^{\phi\phi}-C_\ell^{\phi\phi, linear}/C_\ell^{\phi\phi, linear}$')
#ax[2].set_xlim([1e-3,1])
#ax[2].set_ylim([-75, 10])

Text(0, 0.5, '$C_\\ell^{\\phi\\phi}-C_\\ell^{\\phi\\phi, linear}/C_\\ell^{\\phi\\phi, linear}$')

In [27]:
power_spectrum_linear = np.linspace(log(0.0001),log(50),200)
power_spectrum_Halofit = np.linspace(log(0.0001),log(50),200)

for i in range(len(k)):
    power_spectrum_linear[i] = M.pk_lin(k[i]*h,z_pk)*h**3
    power_spectrum_Halofit[i] = M1.pk_lin(k[i]*h,z_pk)*h**3

l = np.array(cl_linear['ell'][2:])
tt_linear = np.array(cl_linear['tt'][2:])
ee_linear = np.array(cl_linear['ee'][2:])
pp_linear = np.array(cl_linear['pp'][2:])
tt_Halofit = np.array(cl_Halofit['tt'][2:])
ee_Halofit = np.array(cl_Halofit['ee'][2:])
pp_Halofit = np.array(cl_Halofit['pp'][2:])
tt_dmeff_EFT = np.array(cl_EFT['tt'][2:])
ee_dmeff_EFT = np.array(cl_EFT['ee'][2:])
pp_dmeff_EFT = np.array(cl_EFT['pp'][2:])

In [41]:
# Initializing convenience functions pk_mm_real, pk_gg_l0 etc.
#M2.initialize_output(khvec, z_pk, len(khvec))
power_spectrum_dmeff_EFT = M2.pk_mm_real(1.)
testout = [[0 for x in range(42)] for y in range(len(k))]

#for i in range(len(k)):
#    testout[i][0] = k[i]
#    testout[i][41] = M2.pk_lin(k[i]*h,z_pk)*h
#    for j in range(40):
#        testout[i][j+1] = M2.pk(k[i]*h,z_pk)[j]*h

#for i in range(len(k)):
#    for j in range(40):
 #       print(testout[i][j])
    
# Plotting spectra
ax[0].plot(l, (tt_Halofit-tt_linear)/tt_linear * 100, color = 'green', label=r'$halofit$')
ax[0].plot(l, (tt_dmeff_EFT-tt_linear)/tt_linear * 100, color = 'purple', label=r'$EFT$')
ax[0].legend(fontsize='8',ncol=1,loc='upper right')

ax[1].plot(l, (ee_Halofit-ee_linear)/ee_linear * 100, color = 'green', label=r'$halofit$')
ax[1].plot(l, (ee_dmeff_EFT-ee_linear)/ee_linear * 100, color = 'purple', label=r'$EFT$')
ax[1].legend(fontsize='8',ncol=1,loc='upper right')

ax[2].plot(l, (pp_Halofit-pp_linear)/pp_linear * 100, color = 'green', label=r'$halofit$')
ax[2].plot(l, (pp_dmeff_EFT-pp_linear)/pp_linear * 100, color = 'purple', label=r'$EFT$')
ax[2].legend(fontsize='8',ncol=1,loc='upper right')

#ax[3].plot(k, (power_spectrum_Halofit-power_spectrum_linear)/power_spectrum_linear * 100, color = 'green', label=r'$halofit$')
#ax[3].plot(k, (power_spectrum_dmeff_EFT-power_spectrum_linear)/power_spectrum_linear * 100, color = 'purple', label=r'$EFT$')
#ax[3].legend(fontsize='8',ncol=1,loc='upper right')

plot = ax[0].get_figure()
plot.show()

#Cleaning up

M.struct_cleanup()
M.empty()
M1.struct_cleanup()
M1.empty()
M2.struct_cleanup()
M2.empty()

In [ ]:
#Starting CLASS

z_pk = 0.61
common_settings = {# fixed parameters
                   'A_s':2.089e-9,
                   'n_s':0.9649,
                   'tau_reio':0.052,
                   'omega_b':0.02237,
                   'h':0.6736,
                   'YHe':0.2425,
#                   'N_eff':3.046,
                   'N_ur':2.0328,
                   'N_ncdm':1,
#                   'N_ncdm':0,
                   'm_ncdm':0.06,
                   # Interacting Dark Matter parameters
 #                  'sigma_dmeff':1e-25,
 #                  'omega_cdm':1e-15,
 #                  'm_dmeff':1.0,
 #                  'npow_dmeff':0,
 #                  'Vrel_dmeff':0,
 #                  'dmeff_target': 'baryons',
                   # other output and precision parameters
#                    'P_k_max_1/Mpc':100.0,
                   'z_pk':z_pk,
                   'output':'mPk,tCl,pCl,lCl',
                   'lensing':'yes'}

#                   'z_max_pk':20.,

M = Class()
M.set(common_settings)
#compute linear
M.set({ 'non linear':'no',
        'omega_cdm':0.120,
      })

M.compute()

M1 = Class()
M1.set(common_settings)
#compute Halofit
M1.set({'non linear':'Halofit',
        'omega_cdm':0.120,
      })

M1.compute()

#now we compute EFT
M2 = Class()
M2.set(common_settings)
M2.set({'non linear':'PT',
        'IR resummation':'Yes',
        'Bias tracers':'Yes',
        'cb':'No',
        'RSD':'Yes',
        'AP':'Yes',
        'Omfid':'0.31',
        'omega_cdm':0.120,
       })
M2.compute()

#Extracting and plotting spectra

bg_linear = M.get_background()
th_linear = M.get_thermodynamics()
cl_linear = M.lensed_cl()

bg_Halofit = M1.get_background()
th_Halofit = M1.get_thermodynamics()
cl_Halofit = M1.lensed_cl()

bg_EFT = M2.get_background()
th_EFT = M2.get_thermodynamics()
cl_EFT = M2.lensed_cl()

h = M.h()
k = np.linspace(log(0.0001),log(50),200)
k = np.exp(k)
twopi = 2.*math.pi
khvec = k*h

f,ax_list = plt.subplots(1,3,figsize=(9,3),constrained_layout=True)
ax = ax_list.ravel()

ax[0].set_xlabel('$\ell$')
ax[0].set_ylabel('$C_\ell^{TT}-C_\ell^{TT, linear}/C_\ell^{TT, linear}$')
ax[0].set_xlim([1,2500])

ax[1].set_xlabel('$\ell$')
ax[1].set_ylabel('$C_\ell^{EE}-C_\ell^{EE, linear}/C_\ell^{EE linear}$')
ax[1].set_xlim([1,2500])

#ax[2].set_xscale('log')
ax[2].set_xlabel('$\ell$')
#ax[2].set_yscale('log')
ax[2].set_ylabel('$C_\ell^{EE}-C_\ell^{EE, linear}/C_\ell^{EE linear}$')
#ax[2].set_xlim([1e-3,1])
#ax[2].set_ylim([-75, 10])

ax[3].set_xscale('log')
ax[3].set_xlabel('$k$')
# ax[3].set_yscale('log')
ax[3].set_ylabel('$(P-P^{linear})/P^{linear}$')
ax[3].set_xlim([1e-3,1])

In [ ]:
power_spectrum_linear = np.linspace(log(0.0001),log(50),200)
power_spectrum_Halofit = np.linspace(log(0.0001),log(50),200)

for i in range(len(k)):
    power_spectrum_linear[i] = M.pk_lin(k[i]*h,z_pk)*h**3
    power_spectrum_Halofit[i] = M1.pk_lin(k[i]*h,z_pk)*h**3

l = np.array(cl_linear['ell'])
l = np.delete(l, 0)
l = np.delete(l, 0)

tt_linear = np.array(cl_linear['tt'])
tt_linear = np.delete(tt_linear, 0)
tt_linear = np.delete(tt_linear, 0)

ee_linear = np.array(cl_linear['ee'])
ee_linear = np.delete(ee_linear, 0)
ee_linear = np.delete(ee_linear, 0)

pp_linear = np.array(cl_linear['pp'])
pp_linear = np.delete(pp_linear, 0)
pp_linear = np.delete(pp_linear, 0)

tt_Halofit = np.array(cl_Halofit['tt'])
tt_Halofit = np.delete(tt_Halofit, 0)
tt_Halofit = np.delete(tt_Halofit, 0)

ee_Halofit = np.array(cl_Halofit['ee'])
ee_Halofit = np.delete(ee_Halofit, 0)
ee_Halofit = np.delete(ee_Halofit, 0)

pp_Halofit = np.array(cl_Halofit['pp'])
pp_Halofit = np.delete(pp_Halofit, 0)
pp_Halofit = np.delete(pp_Halofit, 0)

tt_dmeff_EFT = np.array(cl_EFT['tt'])
tt_dmeff_EFT = np.delete(tt_dmeff_EFT, 0)
tt_dmeff_EFT = np.delete(tt_dmeff_EFT, 0)

ee_dmeff_EFT = np.array(cl_EFT['ee'])
ee_dmeff_EFT = np.delete(ee_dmeff_EFT, 0)
ee_dmeff_EFT = np.delete(ee_dmeff_EFT, 0)

pp_dmeff_EFT = np.array(cl_EFT['pp'])
pp_dmeff_EFT = np.delete(pp_dmeff_EFT, 0)
pp_dmeff_EFT = np.delete(pp_dmeff_EFT, 0)

##Initializing convenience functions pk_mm_real, pk_gg_l0 etc.

M2.initialize_output(khvec, z_pk, len(khvec))

power_spectrum_dmeff_EFT = M2.pk_mm_real(1.)

testout = [[0 for x in range(42)] for y in range(len(k))]

for i in range(len(k)):
    testout[i][0] = k[i]
    testout[i][41] = M2.pk_lin(k[i]*h,z_pk)*h
    for j in range(40):
        testout[i][j+1] = M2.pk(k[i]*h,z_pk)[j]*h

#for i in range(len(k)):
#    for j in range(40):
 #       print(testout[i][j])

#Plotting spectra

ax[0].plot(l, (tt_Halofit-tt_linear)/tt_linear * 100, color = 'green', label=r'$halofit$')
ax[0].plot(l, (tt_dmeff_EFT-tt_linear)/tt_linear * 100, color = 'purple', label=r'$IDM + EFT$')
ax[0].legend(fontsize='8',ncol=1,loc='upper right')

ax[1].plot(l, (ee_Halofit-ee_linear)/ee_linear * 100, color = 'green', label=r'$halofit$')
ax[1].plot(l, (ee_dmeff_EFT-ee_linear)/ee_linear * 100, color = 'purple', label=r'$IDM + EFT$')
ax[1].legend(fontsize='8',ncol=1,loc='upper right')

ax[2].plot(l, (pp_Halofit-pp_linear)/pp_linear * 100, color = 'green', label=r'$halofit$')
ax[2].plot(l, (pp_dmeff_EFT-pp_linear)/pp_linear * 100, color = 'purple', label=r'$IDM + EFT$')
ax[2].legend(fontsize='8',ncol=1,loc='upper right')

ax[3].plot(k, (power_spectrum_Halofit-power_spectrum_linear)/power_spectrum_linear * 100, color = 'green', label=r'$halofit$')
ax[3].plot(k, (power_spectrum_dmeff_EFT-power_spectrum_linear)/power_spectrum_LCDM * 100, color = 'purple', label=r'$EFT$')
ax[3].legend(fontsize='8',ncol=1,loc='upper right')

plot = ax[0].get_figure()
plot.show()

#Cleaning up

M.struct_cleanup()
M.empty()
M1.struct_cleanup()
M1.empty()
M2.struct_cleanup()
M2.empty()